#1. Load and Explore the Data : We will use two libraries in Python to load and explore our dataset: pandas and sqlalchemy.

#2. Create a SQLite database using SQLAlchemy.
Export each dataframe as a table to the SQLite database.



In [4]:
from google.colab import files
import os, pandas as pd
from sqlalchemy import create_engine, inspect
from pathlib import Path
import re

DATA_DIR = "/content/olist_manual"
DB_PATH  = "/content/olist.sqlite"
os.makedirs(DATA_DIR, exist_ok=True)

# Define required files
required = [
    "olist_customers_dataset.csv","olist_sellers_dataset.csv","olist_order_reviews_dataset.csv",
    "olist_order_items_dataset.csv","olist_products_dataset.csv","olist_geolocation_dataset.csv",
    "product_category_name_translation.csv","olist_orders_dataset.csv","olist_order_payments_dataset.csv",
]

# Check if files are missing
missing = [fn for fn in required if not (Path(DATA_DIR)/fn).exists()]

# Upload files only if they are missing
if missing:
    print(f"The following required files are missing and need to be uploaded: {', '.join(missing)}")
    print("Please upload the listed CSV files now.")
    uploads = files.upload()  # multi-select the 9 CSVs on your computer
    for uploaded_name, data in uploads.items():
        # Remove the '(X)' suffix that Colab sometimes adds if a file of the same name exists
        # This regex matches ' (number)' at the end of the filename before the extension
        target_name = re.sub(r' \((\d+)\)\.csv$', '.csv', uploaded_name)

        with open(os.path.join(DATA_DIR, target_name), "wb") as f:
            f.write(data)

    # Re-verify after upload attempt
    missing = [fn for fn in required if not (Path(DATA_DIR)/fn).exists()]

assert not missing, f"After upload attempt, the following files are still missing: {', '.join(missing)}"

# Create SQLite
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)
mapping = {
    "olist_customers_dataset.csv": "olist_customers",
    "olist_sellers_dataset.csv": "olist_sellers",
    "olist_order_reviews_dataset.csv": "olist_order_reviews",
    "olist_order_items_dataset.csv": "olist_order_items",
    "olist_products_dataset.csv": "olist_products",
    "olist_geolocation_dataset.csv": "olist_geolocation",
    "product_category_name_translation.csv": "product_category_name_translation",
    "olist_orders_dataset.csv": "olist_orders",
    "olist_order_payments_dataset.csv": "olist_order_payments",
}
for fname, tname in mapping.items():
    pd.read_csv(Path(DATA_DIR)/fname).to_sql(tname, con=engine, if_exists="replace", index=False)

print("Tables:", inspect(engine).get_table_names())
pd.read_sql_query("SELECT * FROM olist_customers LIMIT 5;", con=engine)


The following required files are missing and need to be uploaded: olist_customers_dataset.csv, olist_sellers_dataset.csv, olist_order_reviews_dataset.csv, olist_order_items_dataset.csv, olist_geolocation_dataset.csv, product_category_name_translation.csv, olist_orders_dataset.csv, olist_order_payments_dataset.csv
Please upload the listed CSV files now.


Saving olist_customers_dataset.csv to olist_customers_dataset (2).csv
Saving olist_geolocation_dataset.csv to olist_geolocation_dataset (2).csv
Saving olist_order_items_dataset.csv to olist_order_items_dataset (2).csv
Saving olist_order_payments_dataset.csv to olist_order_payments_dataset (2).csv
Saving olist_order_reviews_dataset.csv to olist_order_reviews_dataset (2).csv
Saving olist_orders_dataset.csv to olist_orders_dataset (2).csv
Saving olist_products_dataset.csv to olist_products_dataset (1).csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset (2).csv
Saving product_category_name_translation.csv to product_category_name_translation (2).csv
Tables: ['olist_customers', 'olist_geolocation', 'olist_order_items', 'olist_order_payments', 'olist_order_reviews', 'olist_orders', 'olist_products', 'olist_sellers', 'product_category_name_translation']


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


#Query 1: Count and Percentage of Orders Purchased in Jan 2018 with 5 Review Score

Write and execute a SQL query to count the number of orders purchased in January 2018 that have a review score of 5 and calculate the percentage of such orders.

In [6]:
q1 = '''
SELECT
    SUM(review_score_5)                           AS orders_with_review_5,
    COUNT(*)                                      AS total_orders,
    ROUND(100.0 * SUM(review_score_5) / NULLIF(COUNT(*), 0), 2) AS percentage_5_star

FROM (
    SELECT
        o.order_id,
        CASE WHEN MAX(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) = 1
             THEN 1 ELSE 0 END AS review_score_5
    FROM olist_orders o
    LEFT JOIN olist_order_reviews r
           ON r.order_id = o.order_id
    WHERE date(o.order_purchase_timestamp) >= date('2018-01-01')
      AND date(o.order_purchase_timestamp) <  date('2018-02-01')
    GROUP BY o.order_id
) per_order;
'''
df_q1 = pd.read_sql_query(q1, con=engine)
print(df_q1.head())

   orders_with_review_5  total_orders  percentage_5_star
0                  4077          7269              56.09


#Query 2: Customer Purchase Trend Year-on-Year

Write and execute a SQL query to analyze the customer purchase trend year-on-year.


In [ ]:
q2 = """
WITH yearly AS (
    SELECT
        strftime('%Y', o.order_purchase_timestamp) AS yr,
        COUNT(*) AS orders,
        COUNT(DISTINCT o.customer_id) AS unique_customers
    FROM olist_orders o
    GROUP BY strftime('%Y', o.order_purchase_timestamp)
)
SELECT
    yr,
    orders,
    unique_customers,
    ROUND(100 * (orders - LAG(orders) OVER (ORDER BY yr))
          / NULLIF(LAG(orders) OVER (ORDER BY yr), 0), 2) AS orders_yoy_pct,

  /* NULLIF(value, 0) turns the divisor into NULL if it’s zero, so we avoid division-by-zero errors.
  If the previous year’s orders = 0, the result of the division will be NULL instead of crashing. */

    ROUND(100 * (unique_customers - LAG(unique_customers) OVER (ORDER BY yr))
          / NULLIF(LAG(unique_customers) OVER (ORDER BY yr), 0), 2) AS customers_yoy_pct
FROM yearly
ORDER BY yr;
"""
df_q2 = pd.read_sql_query(q2, engine)
df_q2


,yr,orders,unique_customers,orders_yoy_pct,customers_yoy_pct
0,2016,329,329,NaN,NaN
1,2017,45101,45101,13608.51,13608.51
2,2018,54011,54011,19.76,19.76


In [ ]:
sql = '''
SELECT
    month AS month_no,
    CASE
        WHEN a.month = '01' THEN 'Jan'
        WHEN a.month = '02' THEN 'Feb'
        WHEN a.month = '03' THEN 'Mar'
        WHEN a.month = '04' THEN 'Apr'
        WHEN a.month = '05' THEN 'May'
        WHEN a.month = '06' THEN 'Jun'
        WHEN a.month = '07' THEN 'Jul'
        WHEN a.month = '08' THEN 'Aug'
        WHEN a.month = '09' THEN 'Sep'
        WHEN a.month = '10' THEN 'Oct'
        WHEN a.month = '11' THEN 'Nov'
        WHEN a.month = '12' THEN 'Dec'
        ELSE 0
    END AS month,
    SUM(CASE WHEN a.year = '2016' THEN 1 ELSE 0 END) AS Year2016,
    SUM(CASE WHEN a.year = '2017' THEN 1 ELSE 0 END) AS Year2017,
    SUM(CASE WHEN a.year = '2018' THEN 1 ELSE 0 END) AS Year2018
FROM (
    SELECT
        customer_id,
        order_id,
        order_delivered_customer_date,
        order_status,
        strftime('%Y', order_delivered_customer_date) AS Year,
        strftime('%m', order_delivered_customer_date) AS Month
    FROM olist_orders
    WHERE order_status = 'delivered' AND order_delivered_customer_date IS NOT NULL
    GROUP BY customer_id, order_id, order_delivered_customer_date
    ORDER BY order_delivered_customer_date ASC
) a
GROUP BY month
ORDER BY month_no ASC
'''

df_sql = pd.read_sql_query(sql, con=engine)
print(df_sql.head(12))

   month_no month  Year2016  Year2017  Year2018
0        01   Jan         0       283      6597
1        02   Feb         0      1351      5850
2        03   Mar         0      2382      6824
3        04   Apr         0      1849      7850
4        05   May         0      3751      7111
5        06   Jun         0      3223      6829
6        07   Jul         0      3455      5839
7        08   Aug         0      4302      8314
8        09   Sep         0      3965        56
9        10   Oct       205      4494         3
10       11   Nov        58      4670         0
11       12   Dec         4      7205         0


#Query 3: Average Order Values of Customers
Write and execute a SQL query to calculate the average order values of customers.


In [ ]:
q3 = """
WITH order_values AS (
    SELECT
        oi.order_id,
        SUM(oi.price + oi.freight_value) AS order_value
    FROM olist_order_items oi
    GROUP BY oi.order_id
),
customer_orders AS (
    SELECT
        o.customer_id,
        ov.order_value
    FROM olist_orders o
    JOIN order_values ov ON ov.order_id = o.order_id
)
SELECT
    c.customer_id,
    COUNT(*) AS orders_count,
    ROUND(AVG(order_value), 2) AS avg_order_value,
    ROUND(SUM(order_value), 2) AS total_spent
FROM customer_orders c
GROUP BY c.customer_id
ORDER BY total_spent DESC;  -- optional: see top spenders first
"""
df_q3 = pd.read_sql_query(q3, engine)
df_q3.head(20)  # show a sample


,customer_id,orders_count,avg_order_value,total_spent
0,1617b1357756262bfa56ab541c47bc16,1,13664.08,13664.08
1,ec5b2ba62e574342386871631fafd3fc,1,7274.88,7274.88
2,c6e2731c5b391845f6800c97401a43a9,1,6929.31,6929.31
3,f48d464a0baaea338cb25f816991ab1f,1,6922.21,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,1,6726.66,6726.66
5,05455dfa7cd02f13d132aa7a6a9729c6,1,6081.54,6081.54
6,df55c14d1476a9a3467f131269c2477f,1,4950.34,4950.34
7,e0a2412720e9ea4f26c1ac985f6a7358,1,4809.44,4809.44
8,24bbf5fd2f2e1b359ee7de94defc4a15,1,4764.34,4764.34
9,3d979689f636322c62418b6346b1c6d2,1,4681.78,4681.78


#Query 4: Top 5 Cities with Highest Revenue from 2016 to 2018

Write and execute a SQL query to find the top 5 cities with the highest revenue from 2016 to 2018.


In [8]:
q4 = '''
SELECT
    c.customer_city,
    ROUND(SUM(oi.price + oi.freight_value), 2) AS total_revenue  ----sum of price and freight value
    FROM olist_orders o
JOIN olist_customers c
  ON o.customer_id = c.customer_id
JOIN olist_order_items oi
  ON o.order_id = oi.order_id
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'  ---filter by year range
GROUP BY c.customer_city
ORDER BY total_revenue DESC
LIMIT 5;
'''
df_q4 = pd.read_sql_query(q4, con=engine)
display(df_q4)

,customer_city,total_revenue
0,sao paulo,2170227.12
1,rio de janeiro,1154234.02
2,belo horizonte,416733.39
3,brasilia,352305.14
4,curitiba,244739.87


#Query 5: State Wise Revenue Table Between 2016 to 2018

Write and execute a SQL query to create a state-wise revenue table between 2016 to 2018.


In [9]:
q5 = '''
SELECT
    c.customer_state,
    ROUND(SUM(oi.price + oi.freight_value), 2) AS total_revenue      ----sum of price and freight value
FROM olist_orders o
JOIN olist_customers c
  ON o.customer_id = c.customer_id
JOIN olist_order_items oi
  ON o.order_id = oi.order_id
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'   ---filter by year range
GROUP BY c.customer_state
ORDER BY total_revenue DESC;
'''
df_q5 = pd.read_sql_query(q5, con=engine)
display(df_q5)

,customer_state,total_revenue
0,SP,5921678.12
1,RJ,2129681.98
2,MG,1856161.49
3,RS,885826.76
4,PR,800935.44
5,BA,611506.67
6,SC,610213.60
7,DF,353229.44
8,GO,347706.93
9,ES,324801.91


# Query 6: Top Successful Sellers in Terms of Goods Sold, Revenue, and Customer Count
 Write and execute a SQL query to identify the top successful sellers in terms of the number of goods sold, total revenue, customer count, and sellers with the highest 5-star ratings.

**Part1**:
Calculate the **total number of goods sold by seller**


In [16]:
seller_revenue_goods_query = '''
SELECT
    seller_id,
    COUNT(order_item_id) AS goods_sold,           ---count of order items for goods sold
    ROUND(SUM(price + freight_value), 2) AS total_revenue   ---sum of price and freight value
FROM
    olist_order_items
GROUP BY
    seller_id
ORDER BY
    total_revenue DESC;
'''

df_seller_revenue_goods = pd.read_sql_query(seller_revenue_goods_query, con=engine)
df_seller_revenue_goods.head()

,seller_id,goods_sold,total_revenue
0,4869f7a5dfa277a7dca6462dcf3b52b2,1156,249640.70
1,7c67e1448b00f6e969d365cea6b010ab,1364,239536.44
2,53243585a1d6dc2643021fd1853d8905,410,235856.68
3,4a3ca9315b744ce9f8e9374361493884,1987,235539.96
4,fa1c13f2614d7b5c4749cbc52fecda94,586,204084.73


**Part 2**: Calculate **Seller Unique Customers**


In [11]:
seller_unique_customers_query = '''
SELECT
    oi.seller_id,
    COUNT(DISTINCT o.customer_id) AS unique_customers_count   ---count of unique customers
FROM
    olist_order_items oi
JOIN
    olist_orders o ON oi.order_id = o.order_id
GROUP BY
    oi.seller_id
ORDER BY
    unique_customers_count DESC;
'''

df_seller_unique_customers = pd.read_sql_query(seller_unique_customers_query, con=engine)
df_seller_unique_customers.head()

,seller_id,unique_customers_count
0,6560211a19b47992c3666cc44a7e94c0,1854
1,4a3ca9315b744ce9f8e9374361493884,1806
2,cc419e0650a3c5ba77189a1882b7556a,1706
3,1f50f920176fa81dab994f9023523100,1404
4,da8622b14eb17ae2831f4ac5b9dab84a,1314


**Part 3** Calculate **Seller 5-Star Ratings**


In [12]:
seller_5_star_ratings_query = '''
SELECT
    oi.seller_id,
    COUNT(r.review_score) AS five_star_ratings_count
FROM
    olist_order_items oi
JOIN
    olist_order_reviews r ON oi.order_id = r.order_id
WHERE
    r.review_score = 5
GROUP BY
    oi.seller_id
ORDER BY
    five_star_ratings_count DESC;
'''

df_seller_5_star_ratings = pd.read_sql_query(seller_5_star_ratings_query, con=engine)
df_seller_5_star_ratings.head()

,seller_id,five_star_ratings_count
0,1f50f920176fa81dab994f9023523100,1096
1,cc419e0650a3c5ba77189a1882b7556a,1053
2,6560211a19b47992c3666cc44a7e94c0,1024
3,4a3ca9315b744ce9f8e9374361493884,947
4,da8622b14eb17ae2831f4ac5b9dab84a,893


**Part4**:
Combine these separate dataframes into a single comprehensive dataframe to analyze and summarize the findings for top successful sellers.



In [13]:
seller_data = pd.merge(df_seller_revenue_goods, df_seller_unique_customers, on='seller_id', how='left')
seller_data = pd.merge(seller_data, df_seller_5_star_ratings, on='seller_id', how='left')

# Fill NaN values with 0 for counts/sums where a seller might not have a record in one of the merged tables
seller_data = seller_data.fillna(0)

# Display the combined data for the top sellers
print(seller_data.head())

                          seller_id  goods_sold  total_revenue  \
0  4869f7a5dfa277a7dca6462dcf3b52b2        1156      249640.70   
1  7c67e1448b00f6e969d365cea6b010ab        1364      239536.44   
2  53243585a1d6dc2643021fd1853d8905         410      235856.68   
3  4a3ca9315b744ce9f8e9374361493884        1987      235539.96   
4  fa1c13f2614d7b5c4749cbc52fecda94         586      204084.73   

   unique_customers_count  five_star_ratings_count  
0                    1132                    683.0  
1                     982                    437.0  
2                     358                    210.0  
3                    1806                    947.0  
4                     585                    394.0  


**Part 5**:
To combine all the seller metrics into a single SQL query and identify the top sellers, we use CTEs for each metric (goods sold/revenue, unique customers, and 5-star ratings) and then join them together.



In [17]:
top_sellers_query = '''
WITH SellerRevenueGoods AS (
    SELECT
        seller_id,
        COUNT(order_item_id) AS goods_sold,
        ROUND(SUM(price + freight_value), 2) AS total_revenue
    FROM
        olist_order_items
    GROUP BY
        seller_id
),
SellerUniqueCustomers AS (
    SELECT
        oi.seller_id,
        COUNT(DISTINCT o.customer_id) AS unique_customers_count
    FROM
        olist_order_items oi
    JOIN
        olist_orders o ON oi.order_id = o.order_id
    GROUP BY
        oi.seller_id
),
SellerFiveStarRatings AS (
    SELECT
        oi.seller_id,
        COUNT(r.review_score) AS five_star_ratings_count
    FROM
        olist_order_items oi
    JOIN
        olist_order_reviews r ON oi.order_id = r.order_id
    WHERE
        r.review_score = 5
    GROUP BY
        oi.seller_id
)
SELECT
    srg.seller_id,
    srg.goods_sold,
    srg.total_revenue,
    COALESCE(suc.unique_customers_count, 0) AS unique_customers_count,
    COALESCE(sfv.five_star_ratings_count, 0) AS five_star_ratings_count
FROM
    SellerRevenueGoods srg
LEFT JOIN
    SellerUniqueCustomers suc ON srg.seller_id = suc.seller_id
LEFT JOIN
    SellerFiveStarRatings sfv ON srg.seller_id = sfv.seller_id
ORDER BY
    srg.total_revenue DESC,
    srg.goods_sold DESC,
    unique_customers_count DESC,
    five_star_ratings_count DESC;
'''
df_top_sellers = pd.read_sql_query(top_sellers_query, con=engine)
(df_top_sellers.head(10))

,seller_id,goods_sold,total_revenue,unique_customers_count,five_star_ratings_count
0,4869f7a5dfa277a7dca6462dcf3b52b2,1156,249640.70,1132,683
1,7c67e1448b00f6e969d365cea6b010ab,1364,239536.44,982,437
2,53243585a1d6dc2643021fd1853d8905,410,235856.68,358,210
3,4a3ca9315b744ce9f8e9374361493884,1987,235539.96,1806,947
4,fa1c13f2614d7b5c4749cbc52fecda94,586,204084.73,585,394
5,da8622b14eb17ae2831f4ac5b9dab84a,1551,185192.32,1314,893
6,7e93a43ef30c4f03f38b393420bc753a,340,182754.05,336,213
7,1025f0e2d44d7041d6cf58b6550e0bfa,1428,172860.69,915,729
8,7a67c85e85bb2ce8582c35f2203ad736,1171,162648.38,1160,717
9,955fee9216a65b617aa5c0531780ce60,1499,160602.68,1287,804


## Summary:

### Q&A
1.  **Who are the top successful sellers based on combined metrics?**
    Seller `4a3ca9315b744ce9f8e9374361493884` emerged as an exceptionally well-rounded and highly successful entity, consistently performing at the top across multiple critical metrics. Seller `4869f7a5dfa277a7dca6462dcf3b52b2` was identified as the top earner in terms of total revenue.

2.  **What defines their success across the calculated metrics?**
    Success is defined by a combination of high total revenue, a large volume of goods sold, a significant number of unique customers, and excellent customer satisfaction as measured by 5-star ratings.

### Data Analysis Key Findings
*   Seller `4869f7a5dfa277a7dca6462dcf3b52b2` achieved the highest total revenue of R\$249,640.70.
*   Seller `4a3ca9315b744ce9f8e9374361493884` sold the most goods (1987 units), attracted the most unique customers (1806), and received the highest number of 5-star ratings (947). This seller also had a very high total revenue of R\$235,539.96.
*   The data for each seller was successfully aggregated, combining `goods_sold`, `total_revenue`, `unique_customers_count`, and `five_star_ratings_count` into a single view, with `COALESCE` handling missing values by defaulting them to 0.

### Insights or Next Steps
*   Further investigation into seller `4a3ca9315b744ce9f8e9374361493884`'s product catalog, pricing strategy, and customer engagement could reveal best practices applicable to other sellers on the platform.
*   Analyze the product categories and pricing strategies of top revenue-generating sellers versus high-volume sellers to understand different success models.


#Query 7: Delivery Success Rate Across States


Write and execute a SQL query to calculate the delivery success rate across different states.

**Part 1**:
 CTE to prepare the data, then join it with the customers table to calculate the success rate by state.



In [20]:
q7 = '''
WITH StateOrderCounts AS (
    SELECT
        c.customer_state,
        COUNT(o.order_id) AS total_orders,   ---total orders are order_id counts
        COUNT(CASE WHEN o.order_status = 'delivered' THEN o.order_id ELSE NULL END) AS delivered_orders   ---isolated only delivered orders
    FROM
        olist_orders o
    JOIN
        olist_customers c ON o.customer_id = c.customer_id
    GROUP BY
        c.customer_state           ---grouped by customer_state to perform calculations for each state individually
)
SELECT
    customer_state,
    total_orders,
    delivered_orders,
    ROUND(CAST(delivered_orders AS REAL) * 100 / NULLIF(total_orders, 0), 2) AS delivery_success_rate   ---conditionally counts only those order IDs where the order_status is 'delivered'. If the status is anything else, NULL is returned for that order_id, and COUNT() ignores NULL values.
FROM
    StateOrderCounts
ORDER BY
    delivery_success_rate DESC;
'''
df_delivery_success = pd.read_sql_query(q7, con=engine)
print(df_delivery_success.head())

  customer_state  total_orders  delivered_orders  delivery_success_rate
0             AC            81                80                  98.77
1             AP            68                67                  98.53
2             ES          2033              1995                  98.13
3             MS           715               701                  98.04
4             AM           148               145                  97.97


#Query 8: Preferred Form of Payment for Different Categories

Write and execute a SQL query to find the preferred form of payment for different product categories.

In [19]:
q8 = '''
WITH CategoryPaymentCounts AS (
    SELECT
        pct.product_category_name_english AS product_category,
        op.payment_type,
        COUNT(op.payment_type) AS payment_count
    FROM olist_order_payments op
    JOIN olist_orders o ON op.order_id = o.order_id
    JOIN olist_order_items oi ON o.order_id = oi.order_id
    JOIN olist_products p ON oi.product_id = p.product_id
    LEFT JOIN product_category_name_translation pct ON p.product_category_name = pct.product_category_name
    GROUP BY
        pct.product_category_name_english,
        op.payment_type
),
RankedPayments AS (            ---ranks the payment types within each product category based on their payment_count
    SELECT
        product_category,
        payment_type,
        payment_count,
        ROW_NUMBER() OVER(PARTITION BY product_category ORDER BY payment_count DESC) AS rn   ---data into separate groups for each product_category
    FROM CategoryPaymentCounts
)
SELECT
    product_category,
    payment_type AS preferred_payment_method,
    payment_count
FROM RankedPayments
WHERE rn = 1
ORDER BY
    payment_count DESC;
'''
df_q8 = pd.read_sql_query(q8, con=engine)
display(df_q8.head())

,product_category,preferred_payment_method,payment_count
0,bed_bath_table,credit_card,8959
1,health_beauty,credit_card,7566
2,sports_leisure,credit_card,6635
3,furniture_decor,credit_card,6379
4,computers_accessories,credit_card,5436


#Query 9: Distance Between Cities

Write and execute a SQL query to calculate the distance between cities.


## Calculate City Centroids

### Subtask:
Calculate the geographical centroid (average latitude and longitude) for each city present in the `olist_geolocation` dataset. This step aggregates the multiple geographical entries for each city into a single representative coordinate. The result will be stored in a DataFrame.


**Reasoning**:
To calculate the city centroids, I need to first retrieve the raw geolocation data from the `olist_geolocation` table using a SQL query, then load it into a Pandas DataFrame, and finally group by city to calculate the mean latitude and longitude.



In [21]:
q_geolocation = '''
SELECT
    geolocation_city,
    geolocation_lat,
    geolocation_lng
FROM
    olist_geolocation;
'''
df_geolocation = pd.read_sql_query(q_geolocation, con=engine)

city_centroids = df_geolocation.groupby('geolocation_city').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()
city_centroids.rename(columns={'geolocation_lat': 'city_lat', 'geolocation_lng': 'city_lng'}, inplace=True)

print(city_centroids.head())


     geolocation_city   city_lat   city_lng
0            * cidade -25.571748 -49.334374
1  ...arraial do cabo -22.969370 -42.029834
2      4o. centenario -24.271860 -53.069433
3       4º centenario -24.279801 -53.074723
4     abadia de goias -16.767063 -49.438089


## Calculate Simplified Distance Between Cities

### Subtask:
Using the calculated city centroids, write a SQL query to calculate a simplified Euclidean distance between a few selected cities (e.g., 'sao paulo' and 'rio de janeiro'). This query will demonstrate how to compare the proximity of cities directly within SQL, acknowledging that this is an approximation and not a precise geodetic (Haversine) distance. The result will show these simplified distances.


**Reasoning**:
First, I will save the `city_centroids` DataFrame to the SQLite database so that it can be queried using SQL in the next step. This allows the SQL query to access the pre-calculated city centroids.



In [22]:
city_centroids.to_sql('city_centroids', con=engine, if_exists='replace', index=False)
print("DataFrame 'city_centroids' saved to SQLite database.")

DataFrame 'city_centroids' saved to SQLite database.


**Reasoning**:
Now that the `city_centroids` DataFrame is saved to the SQLite database, I will write and execute a SQL query to calculate the Euclidean distance between 'sao paulo' and 'rio de janeiro' using their average latitudes and longitudes, and then display the result.



In [23]:
distance_query = '''
SELECT
    SQRT(
        POWER(sp.city_lat - rj.city_lat, 2) +
        POWER(sp.city_lng - rj.city_lng, 2)
    ) AS euclidean_distance
FROM
    city_centroids sp
JOIN
    city_centroids rj
ON
    sp.geolocation_city = 'sao paulo' AND rj.geolocation_city = 'rio de janeiro';
'''
df_distance = pd.read_sql_query(distance_query, con=engine)
print(df_distance.head())

   euclidean_distance
0            3.384532


## Summary:

### Data Analysis Key Findings

*   **City Centroids Calculated**: Geographical centroids (average latitude and longitude) were successfully calculated for all unique cities present in the `olist_geolocation` dataset. This aggregated multiple geographical entries into a single representative coordinate for each city.
*   **Simplified Distance Calculated**: A simplified Euclidean distance of `3.384532` was calculated between 'sao paulo' and 'rio de janeiro' using their respective city centroids. This calculation was performed directly within SQL.

### Insights or Next Steps

*   **Accuracy Limitations**: The Euclidean distance method used for calculating distances between geographical points is a simplified approximation. It treats latitude and longitude as Cartesian coordinates on a flat plane, which is inaccurate for larger distances or points closer to the poles, as the Earth is a sphere.
*   **More Precise Methods**: For higher accuracy in geographical distance calculations, consider using geodetic methods such as the Haversine formula or great-circle distance. These can be implemented using Python libraries like `geopy` or `haversine`, which account for the Earth's curvature, providing more realistic distance measurements.
